<a href="https://colab.research.google.com/github/thiagoudweb/Projeto-RAg---1a-VA/blob/main/RAGIPOB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!pip install langchain langchain-community langchain-openai langchain-groq
!pip install streamlit streamlit_chat
!pip install sentence-transformers
!pip install chromadb
!pip install pypdf unstructured
!pip install PyPDF2
!pip install streamlit
!pip install -U langchain langchain-openai
!pip install faiss-cpu transformers groq
!npm install -g localtunnel






⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 3s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

In [20]:
# @title bibliotecas essenciaiss
import langchain
import streamlit as st
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import os
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.chat_models import ChatOpenAI
import transformers
import sentence_transformers
import groq
from langchain.schema import HumanMessage




In [21]:
# @title extraindo os textos do pdf
import PyPDF2

def carregar_texto_pdf(caminho_pdf):
    texto_completo = ""
    with open(caminho_pdf, "rb") as arquivo:
        leitor = PyPDF2.PdfReader(arquivo)
        for pagina in range(len(leitor.pages)):
            texto_completo += leitor.pages[pagina].extract_text() + "\n"
    return texto_completo


In [22]:
# @title Carregar pdf de maneira manual
from google.colab import files

uploaded = files.upload()  #upload do arquivo manual
caminho_pdf = list(uploaded.keys())[0]

texto_prospecto = carregar_texto_pdf(caminho_pdf)
print(texto_prospecto[:1000])


Saving book.pdf to book.pdf
As informações contidas neste Prospecto Preliminar estão sob análise da Comissão de Valores Mobiliários, a qual ainda não se man ifestou a seu respeito. O presente Prospecto Preliminar está sujeito à complementação e correção. O Prospecto Definitivo estará  disponível nas páginas da rede mundial de computadores da Companhia; das
Instituições Participantes da Oferta; das entidades administradoras de mercado organizado de valores mobiliários onde os valore s mobiliários da Companhia sejam admitidos à negociação; e da CVM.  ESTE DOCUMENTO É UMA MINUTA INICIAL SUJEITA A ALTERAÇÕES E COMPLEMENTAÇÕES, TENDO SIDO ARQUIVADO NA COMISSÃO DE VALORES MOBILIÁ RIOS PARA FINS EXCLUSIVOS DE ANÁLISE E EXIGÊNCIAS 
POR PARTE DESSA AUTARQUIA. ESTE DOCUMENTO, PO RTANTO, NÃO SE CARACTERIZA COMO O PROSPECTO PRELIMINAR DA OFERTA E NÃO CONSTITUI U MA OFERTA DE VENDA OU UMA SOLICITAÇÃO PARA 
OFERTA DE COMPRA DE TÍTULOS E VALORES MOBILIÁRIOS NO BRASIL, NOS ESTADOS UNIDOS DA AMÉRICA OU

In [47]:
# @title Dividindo texto em chunnks


def dividir_texto(texto, tamanho_chunk, sobreposicao):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=tamanho_chunk,
        chunk_overlap=sobreposicao,
        separators=["\n\n", "\n", " ", ""]
    )
    return splitter.split_text(texto)




In [51]:
# @title Salvando vetor de chunnks

# CONSTANTES PARA PARAMETRICAR OS CHUNKS
tamanhochunk = 500
sobreposicao = 100

chunks = dividir_texto(texto_prospecto,tamanhochunk,sobreposicao)

In [ ]:
# @title Teste com diferentes modelos de embeddings
from sentence_transformers import SentenceTransformer

# modelos hugging
model_names = [
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/multi-qa-mpnet-base-dot-v1",
    "intfloat/e5-large-v2"
]


texto_exemplo = "ste Prospecto (“Acionista Vendedor Pessoa Física”, em conjunto com o Prisma Bazar FIP, “Acionistas Vendedores”) estão realizando uma oferta pública de distribuição primária e secundária de, inicialmente, [•] ações ordinárias, nominativas, "
# vetor para armazenar embeddings
embeddings = {}

for model_name in model_names:
    model = SentenceTransformer(model_name)
    embedding = model.encode(texto_exemplo)
    embeddings[model_name] = embedding
    print(f"Modelo: {model_name} - Tamanho do vetor: {len(embedding)}")


print("\nexemplo de embedding gerado (valores iniciais):")
print(embeddings[model_names[0]][:10])




In [50]:
# @title Transformando chunks em embeddings

# modelo de embeddings usando lib hugging Face
modelo_embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# criar embeddings para os chunk de textos
vetores_texto = [modelo_embedding.embed_query(chunk) for chunk in chunks]

# MODELOS PARA TESTAR

# sentence-transformers/all-MiniLM-L6-v2 | Rápido e eficiente, bom para buscas gerais.
# sentence-transformers/multi-qa-mpnet-base-dot-v1 | Treinado para perguntas e respostas, pode melhorar a recuperação.
# intfloat/e5-large-v2 | Modelo maior e mais preciso, mas mais pesado.



In [27]:
# @title Banco de dados para guardar os embeddings usando chorma db.

chroma_db = Chroma(persist_directory="./chroma_db", embedding_function=modelo_embedding)

# add textos e embeddinss ao banco
ids = [f"chunk_{i}" for i in range(len(chunks))]  # criando identificação para chunk
chroma_db.add_texts(texts=chunks, ids=ids)

print("Base de dados vetorial criada e populada com sucesso!")


<ipython-input-27-21b8586cb98f>:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_db = Chroma(persist_directory="./chroma_db", embedding_function=modelo_embedding)


Base de dados vetorial criada e populada com sucesso!


In [36]:
# @title Integrando o groq no RAG
import os
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = "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"



# criar o modelo de ia com llama
modelo_groq = ChatGroq(model_name="llama3-8b-8192")



In [29]:
def responder_pergunta_gpt(pergunta, chroma_db, modelo_groq, k=3):
    # Busca os trechos mais relevantes no banco vetorial
    trechos_relevantes = chroma_db.similarity_search(pergunta, k=k)

    # criar o contexto com os embbedings enctrados
    contexto = "\n".join([trecho.page_content for trecho in trechos_relevantes])

    # criar um prompt para o gpt4
    prompt = f"""
    você é um especialista em mercado financeiro e vai usar as informações abaixo para responder a pergunta do usuário com base nos documentos oficiais dos prospectos de ipo.

    CONTEXTO:
    {contexto}

    PERGUNTA:
    {pergunta}

    responda de forma clara,objetiva e baseada apenas no contexto acima.
    """

    # gera a resposta com llama
    resposta = modelo_groq.invoke(prompt)

    return resposta.content  # retornar o texto gerado


In [52]:
pergunta_teste = input()
resposta = responder_pergunta_gpt(pergunta_teste, chroma_db, modelo_groq)

print("Resposta gerada pelo sistema:")
print(resposta)


quem é o ceo?
Resposta gerada pelo sistema:
Segundo o contexto fornecido, o CEO (Chief Executive Officer) é Maurício Barbosa.
